In [ ]:
%run includes/4-storage-setup.ipynb

# Storage
Summary and detail reports on storage buckets and objects deployed across the GCP estate.

In [ ]:

# get buckets
queries = [
    f"""
    SELECT 
    name,
    '{project}' as project,
    location,
    locationType,
    storageClass,
    defaultEventBasedHold,
    labels,
    satisfiesPZS,
    acl,
    customPlacementConfig,
    versioning,
    billing,
    lifecycle,	
    encryption,
    website,
    retentionPolicy,
    autoclass,
    rpo,
    cors,
    updated,
    metageneration,
    defaultObjectAcl,
    iamConfiguration,
    logging
    FROM google.storage.buckets WHERE project = '{project}'
    """
    for project in all_projects
]
buckets_df = run_stackql_queries(queries)
storage_projects = list(buckets_df['project'].unique())
buckets = list(buckets_df['name'].unique())

# public access enforcement
public_access_queries = [
    f"""
    SELECT name, 
    JSON_EXTRACT(iamConfiguration, '$.publicAccessPrevention') as publicAccessPrevention
    FROM google.storage.buckets WHERE project = '{project}'
    """
    for project in all_projects
]
public_access_df = run_stackql_queries(public_access_queries)

# get object summary
object_queries = [
    f"""
    SELECT COUNT(id) as object_count,
    '{bucket}' as bucket
    FROM google.storage.objects WHERE bucket = '{bucket}'
    """
    for bucket in buckets
]
object_counts_df = run_stackql_queries(object_queries)

## Storage Summary

In [ ]:

# show storage counts
show_storage_summary_cards(buckets_df, object_counts_df, public_access_df)

## Bucket and Object Inventory

In [ ]:

# bucket counts
plot_buckets_per_project(buckets_df)

In [ ]:

# object counts
plot_object_counts(object_counts_df, 20)

## Bucket Details

In [ ]:

# show buckets
buckets_df